In [1]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='mC9MOAZs2JOS7JCXOiJavBQ6bAYpqDg8MZrkRXM_J4IV',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.au-syd.cloud-object-storage.appdomain.cloud')

bucket = 'gnnml-donotdelete-pr-a7n66qygvumdpb'
object_key = 'Train_data.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head(10)


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
5,0,tcp,private,REJ,0,0,0,0,0,0,...,19,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,anomaly
6,0,tcp,private,S0,0,0,0,0,0,0,...,9,0.04,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
7,0,tcp,private,S0,0,0,0,0,0,0,...,15,0.06,0.07,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
8,0,tcp,remote_job,S0,0,0,0,0,0,0,...,23,0.09,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
9,0,tcp,private,S0,0,0,0,0,0,0,...,13,0.05,0.06,0.00,0.00,1.00,1.00,0.00,0.00,anomaly


In [3]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.4 MB/s eta 0:00:00


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch_geometric.nn import GINConv, GCNConv, global_add_pool, global_mean_pool, global_max_pool
from torch_geometric.data import Data, Batch
from torch_geometric.utils import dropout_adj, add_remaining_self_loops
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, f1_score, accuracy_score, precision_score, recall_score
import networkx as nx
from tqdm import tqdm
import warnings
import random
import hashlib
import json
import time

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Determine the device to use (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize a dictionary to store training history and metrics
training_history = {
    'pretrain_losses': [],
    'pretrain_contrastive_losses': [],
    'pretrain_reg_losses': [],
    'pretrain_learning_rates': [],
    'finetune_losses': [],
    'finetune_accuracies': [],
    'finetune_learning_rates': [],
    'validation_metrics': [],
    'test_metrics': {},
    'epoch_times': [],
    'batch_losses': [],
    'class_distributions': {},
    'model_parameters': {},
    'training_config': {}
}

Using device: cpu


In [7]:
class NetworkGraphDataset(Dataset):
    def __init__(self, graphs, labels=None, transform=None, scaler=None, fit_scaler=False):
        # Initialize the dataset with graphs, optional labels, transformations, and a scaler
        self.graphs = graphs
        self.labels = labels
        self.transform = transform
        self.scaler = scaler

        # Fit the scaler if requested (typically on the training data)
        if fit_scaler and scaler is not None:
            all_features = []
            for graph in graphs:
                all_features.append(graph.x.numpy())
            all_features = np.vstack(all_features)
            self.scaler.fit(all_features)

    def __len__(self):
        # Return the total number of graphs in the dataset
        return len(self.graphs)

    def __getitem__(self, idx):
        # Get a graph and its corresponding label (if available) by index
        graph = self.graphs[idx].clone()

        # Apply feature scaling if a scaler is provided
        if self.scaler is not None:
            graph.x = torch.FloatTensor(self.scaler.transform(graph.x.numpy()))

        # Apply graph transformations (e.g., augmentations)
        if self.transform:
            graph = self.transform(graph)

        # Return the graph and label, or just the graph if no labels
        if self.labels is not None:
            return graph, self.labels[idx]
        return graph

In [8]:
class GraphAugmentation:
    def __init__(self, node_drop_rate=0.01, edge_drop_rate=0.02, feature_mask_rate=0.05, temporal_noise_std=0.02):
        # Initialize augmentation parameters
        self.node_drop_rate = node_drop_rate
        self.edge_drop_rate = edge_drop_rate
        self.feature_mask_rate = feature_mask_rate
        self.temporal_noise_std = temporal_noise_std

    def structural_augment(self, graph):
        # Apply structural augmentations (node and edge dropping)
        aug_graph = graph.clone()

        # Node dropping
        if aug_graph.num_nodes > 2 and torch.rand(1, device=aug_graph.x.device) < 0.5:
            num_drop = max(1, int(aug_graph.num_nodes * self.node_drop_rate))
            keep_nodes = torch.randperm(aug_graph.num_nodes, device=aug_graph.x.device)[:-num_drop]

            node_mask = torch.zeros(aug_graph.num_nodes, dtype=torch.bool, device=aug_graph.x.device)
            node_mask[keep_nodes] = True

            edge_mask = node_mask[aug_graph.edge_index[0]] & node_mask[aug_graph.edge_index[1]]
            aug_graph.edge_index = aug_graph.edge_index[:, edge_mask]
            if hasattr(aug_graph, 'edge_attr') and aug_graph.edge_attr is not None:
                aug_graph.edge_attr = aug_graph.edge_attr[edge_mask]

            node_mapping = torch.cumsum(node_mask, dim=0) - 1
            aug_graph.edge_index = node_mapping[aug_graph.edge_index]
            aug_graph.x = aug_graph.x[node_mask]

            if hasattr(aug_graph, 'batch'):
                aug_graph.batch = aug_graph.batch[node_mask]

        # Edge dropping
        if aug_graph.edge_index.size(1) > 0:
            aug_graph.edge_index, edge_attr_temp = dropout_adj(
                aug_graph.edge_index,
                aug_graph.edge_attr if hasattr(aug_graph, 'edge_attr') else None,
                p=self.edge_drop_rate,
                training=True
            )
            if hasattr(aug_graph, 'edge_attr') and edge_attr_temp is not None:
                aug_graph.edge_attr = edge_attr_temp

        return aug_graph

    def attribute_augment(self, graph):
        # Apply attribute augmentations (feature masking and temporal noise)
        aug_graph = graph.clone()

        # Feature masking
        mask = torch.rand(aug_graph.x.shape, device=aug_graph.x.device) < self.feature_mask_rate
        aug_graph.x = aug_graph.x.clone()
        aug_graph.x[mask] = 0

        # Temporal noise to edge attributes
        if hasattr(aug_graph, 'edge_attr') and aug_graph.edge_attr is not None:
            temporal_indices = [0, 1] if aug_graph.edge_attr.size(1) > 1 else [0]
            noise = torch.randn(aug_graph.edge_attr[:, temporal_indices].shape, device=aug_graph.edge_attr.device) * self.temporal_noise_std
            aug_graph.edge_attr = aug_graph.edge_attr.clone()
            aug_graph.edge_attr[:, temporal_indices] += noise

        return aug_graph

    def __call__(self, graph):
        # Randomly choose between structural and attribute augmentation
        if torch.rand(1, device=graph.x.device) < 0.5:
            return self.structural_augment(graph)
        else:
            return self.attribute_augment(graph)

In [9]:
class GraphEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=3, dropout=0.1):
        super().__init__()
        self.num_layers = num_layers
        self.dropout = dropout

        # GINConv layers with batch normalization
        self.convs = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        for i in range(num_layers):
            if i == 0:
                conv = GINConv(nn.Sequential(
                    nn.Linear(input_dim, hidden_dim),
                    nn.ReLU(),
                    nn.Linear(hidden_dim, hidden_dim)
                ))
            else:
                conv = GINConv(nn.Sequential(
                    nn.Linear(hidden_dim, hidden_dim),
                    nn.ReLU(),
                    nn.Linear(hidden_dim, hidden_dim)
                ))
            self.convs.append(conv)
            self.batch_norms.append(nn.BatchNorm1d(hidden_dim))

        # Attention mechanism for graph pooling
        self.pool_attention = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, 1)
        )

        # Projection head for contrastive learning
        self.projection_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 2)
        )

    def forward(self, x, edge_index, batch, return_node_embeddings=False):
        h = x

        # Pass through GIN layers
        for i in range(self.num_layers):
            h_new = self.convs[i](h, edge_index)
            h_new = self.batch_norms[i](h_new)
            h_new = F.relu(h_new)
            h_new = F.dropout(h_new, p=self.dropout, training=self.training)

            # Residual connection
            if i > 0:
                h = h + h_new
            else:
                h = h_new

        # Optionally return node embeddings directly
        if return_node_embeddings:
            return h

        # Apply attention pooling to get graph-level embedding
        attention_weights = self.pool_attention(h)
        attention_weights = F.softmax(attention_weights, dim=0)

        graph_embedding = global_add_pool(h * attention_weights, batch)

        # Pass through projection head for contrastive learning
        return self.projection_head(graph_embedding)

In [10]:
class ContrastiveLoss(nn.Module):
    def __init__(self, temperature=0.1, lambda_reg=0.01):
        super().__init__()
        # Initialize temperature and regularization lambda for the loss
        self.temperature = temperature
        self.lambda_reg = lambda_reg

    def forward(self, z_a, z_b):
        # Compute the contrastive loss between two augmented views of the same graph
        batch_size = z_a.size(0)

        # Normalize embeddings
        z_a = F.normalize(z_a, dim=1)
        z_b = F.normalize(z_b, dim=1)

        # Calculate similarity matrix
        similarity_matrix = torch.matmul(z_a, z_b.T) / self.temperature

        # Create labels for contrastive loss (diagonal elements are positive pairs)
        labels = torch.arange(batch_size).to(z_a.device)

        # Compute cross-entropy loss for both directions (a to b and b to a)
        loss_a = F.cross_entropy(similarity_matrix, labels)
        loss_b = F.cross_entropy(similarity_matrix.T, labels)

        contrastive_loss = (loss_a + loss_b) / 2

        # Compute regularization loss (variance of embeddings around their mean)
        z_mean = (z_a + z_b) / 2
        mean_embedding = z_mean.mean(dim=0)
        regularization_loss = torch.norm(z_mean - mean_embedding.unsqueeze(0), dim=1).mean()

        # Combine contrastive and regularization losses
        total_loss = contrastive_loss + self.lambda_reg * regularization_loss

        return total_loss, contrastive_loss, regularization_loss

In [11]:
class ClassificationHead(nn.Module):
    def __init__(self, input_dim, num_classes, dropout=0.3):
        super().__init__()
        # Define a multi-layer perceptron for classification
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, input_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(input_dim // 2, input_dim // 4),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(input_dim // 4, num_classes)
        )

    def forward(self, x):
        # Pass input through the classifier
        return self.classifier(x)

In [12]:
def create_connection_hash(row):
    # Create a hash for unique connection identification
    connection_str = f"{row['protocol_type']}_{row['service']}_{row['flag']}"
    return int(hashlib.md5(connection_str.encode()).hexdigest()[:8], 16) % 10000

def create_graph_from_connection(row, node_features_dim=10):
    # Define node roles
    src_node = 0
    dst_node = 1
    protocol_node = 2
    service_node = 3

    num_nodes = 4

    # Initialize node features
    node_features = torch.zeros(num_nodes, node_features_dim)

    # Populate node features based on connection attributes
    node_features[src_node, 0] = row['src_bytes'] / (row['src_bytes'] + row['dst_bytes'] + 1e-8)
    node_features[src_node, 1] = row['count']
    node_features[src_node, 2] = row['srv_count']
    node_features[src_node, 3] = row['same_srv_rate']
    node_features[src_node, 4] = row['diff_srv_rate']

    node_features[dst_node, 0] = row['dst_bytes'] / (row['src_bytes'] + row['dst_bytes'] + 1e-8)
    node_features[dst_node, 1] = row['dst_host_count']
    node_features[dst_node, 2] = row['dst_host_srv_count']
    node_features[dst_node, 3] = row['dst_host_same_srv_rate']
    node_features[dst_node, 4] = row['dst_host_diff_srv_rate']

    node_features[protocol_node, 0] = 1.0 if row['protocol_type'] == 'tcp' else 0.5 if row['protocol_type'] == 'udp' else 0.0
    node_features[protocol_node, 1] = row['duration']
    node_features[protocol_node, 2] = row['wrong_fragment']
    node_features[protocol_node, 3] = row['urgent']

    node_features[service_node, 0] = row['hot']
    node_features[service_node, 1] = row['num_failed_logins']
    node_features[service_node, 2] = row['logged_in']
    node_features[service_node, 3] = row['num_compromised']

    # Add additional features to all nodes
    additional_features = torch.tensor([
        row['serror_rate'], row['rerror_rate'], row['srv_serror_rate'],
        row['srv_rerror_rate'], row['dst_host_serror_rate'], row['dst_host_rerror_rate']
    ])

    for i in range(min(6, node_features_dim - 5)):
        if i < len(additional_features):
            node_features[:, 5+i] = additional_features[i]

    # Define edges between nodes
    edge_index = torch.tensor([
        [src_node, dst_node, protocol_node, service_node, dst_node, src_node],
        [dst_node, src_node, src_node, dst_node, protocol_node, service_node]
    ], dtype=torch.long)

    # Define edge attributes (e.g., bytes transferred, duration)
    edge_attr = torch.tensor([
        [row['src_bytes'], row['duration']],
        [row['dst_bytes'], row['duration']],
        [row['count'], row['srv_count']],
        [row['srv_count'], row['count']],
        [row['dst_host_count'], row['dst_host_srv_count']],
        [row['dst_host_srv_count'], row['dst_host_count']]
    ], dtype=torch.float)

    # Return a PyTorch Geometric Data object
    return Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr)

In [13]:
def evaluate_model(encoder, classification_head, data_loader, device):
    # Set models to evaluation mode
    encoder.eval()
    classification_head.eval()
    
    predictions = []
    targets = []
    probs = []
    
    with torch.no_grad():
        for batch_data, batch_labels in data_loader:
            batch_data = batch_data.to(device)
            batch_labels = batch_labels.to(device)
            
            # Get embeddings from the encoder
            embeddings = encoder(batch_data.x, batch_data.edge_index, batch_data.batch)
            # Get classification outputs from the classification head
            outputs = classification_head(embeddings)
            
            # Calculate probabilities and predicted classes
            prob = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            # Store predictions, targets, and probabilities
            predictions.extend(predicted.cpu().numpy())
            targets.extend(batch_labels.cpu().numpy())
            probs.extend(prob[:, 1].cpu().numpy())
    
    predictions = np.array(predictions)
    targets = np.array(targets)
    probs = np.array(probs)
    
    # Calculate various classification metrics
    metrics = {
        'accuracy': accuracy_score(targets, predictions),
        'precision': precision_score(targets, predictions, average='weighted'),
        'recall': recall_score(targets, predictions, average='weighted'),
        'f1': f1_score(targets, predictions, average='weighted'),
        'auc': roc_auc_score(targets, probs)
    }
    
    return metrics, predictions, targets, probs

In [14]:
print("Loading and preprocessing data...")

print(f"Dataset shape: {df.shape}")
# Display initial class distribution
class_counts = df['class'].value_counts()
print(f"Class distribution:\n{class_counts}")

training_history['class_distributions']['original'] = class_counts.to_dict()

# Create a unique connection ID hash for data leakage checks
df['connection_id'] = df.apply(create_connection_hash, axis=1)

# Encode categorical columns
categorical_columns = ['protocol_type', 'service', 'flag']
le_dict = {}

for col in categorical_columns:
    le = LabelEncoder()
    df[col + '_encoded'] = le.fit_transform(df[col].astype(str))
    le_dict[col] = le

# Convert multi-class labels to binary (normal vs. anomaly)
binary_class = (df['class'] != 'normal').astype(int)
training_history['class_distributions']['binary'] = {
    'normal': int((binary_class == 0).sum()),
    'anomaly': int((binary_class == 1).sum())
}

# Perform chronological data splitting
chronological_split_point = int(len(df) * 0.6)
train_indices = list(range(chronological_split_point))
test_indices = list(range(chronological_split_point, len(df)))

print(f"Chronological split - Train: {len(train_indices)}, Test: {len(test_indices)}")

# Further split the training data into pre-training and fine-tuning sets
pretrain_size = int(len(train_indices) * 0.7)
finetune_size = len(train_indices) - pretrain_size

pretrain_indices = train_indices[:pretrain_size]
finetune_indices = train_indices[pretrain_size:]

print(f"Pretrain size: {len(pretrain_indices)}, Finetune size: {len(finetune_indices)}")

# Store dataset split information in training history
training_history['dataset_splits'] = {
    'total_samples': len(df),
    'pretrain_size': len(pretrain_indices),
    'finetune_size': len(finetune_indices),
    'test_size': len(test_indices)
}

print("Creating graphs...")
all_graphs = []
all_labels = []

# Iterate through the DataFrame to create a graph for each connection
progress_bar = tqdm(range(len(df)), desc="Creating graphs")
for i in progress_bar:
    row = df.iloc[i]
    graph = create_graph_from_connection(row)
    all_graphs.append(graph)
    all_labels.append(binary_class.iloc[i])

    if i % 1000 == 0:
        progress_bar.set_description(f"Created {i+1}/{len(df)} graphs")

# Separate graphs and labels into respective datasets
pretrain_graphs = [all_graphs[i] for i in pretrain_indices]
finetune_graphs = [all_graphs[i] for i in finetune_indices]
finetune_labels = [all_labels[i] for i in finetune_indices]
test_graphs = [all_graphs[i] for i in test_indices]
test_labels = [all_labels[i] for i in test_indices]

print(f"Pretrain graphs: {len(pretrain_graphs)}")
print(f"Finetune graphs: {len(finetune_graphs)}, Labels: {len(finetune_labels)}")
print(f"Test graphs: {len(test_graphs)}, Labels: {len(test_labels)}")

# Initialize StandardScaler for feature normalization and GraphAugmentation
scaler = StandardScaler()
augmentation = GraphAugmentation()

# Create PyTorch Geometric datasets
pretrain_dataset = NetworkGraphDataset(pretrain_graphs, transform=None, scaler=scaler, fit_scaler=True)
finetune_dataset = NetworkGraphDataset(finetune_graphs, finetune_labels, scaler=scaler)
test_dataset = NetworkGraphDataset(test_graphs, test_labels, scaler=scaler)

# Define model and training parameters
input_dim = pretrain_graphs[0].x.size(1)
hidden_dim = 128
num_epochs_pretrain = 50
num_epochs_finetune = 30
batch_size = 32
learning_rate = 0.001

# Store training configuration
training_history['training_config'] = {
    'input_dim': input_dim,
    'hidden_dim': hidden_dim,
    'num_epochs_pretrain': num_epochs_pretrain,
    'num_epochs_finetune': num_epochs_finetune,
    'batch_size': batch_size,
    'learning_rate': learning_rate,
    'device': str(device)
}

# Create DataLoaders for each dataset
pretrain_loader = DataLoader(pretrain_dataset, batch_size=batch_size, shuffle=True,
                           collate_fn=lambda x: Batch.from_data_list(x), drop_last=True)
finetune_loader = DataLoader(finetune_dataset, batch_size=batch_size, shuffle=True,
                           collate_fn=lambda x: (Batch.from_data_list([item[0] for item in x]),
                                                torch.tensor([item[1] for item in x])), drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                        collate_fn=lambda x: (Batch.from_data_list([item[0] for item in x]),
                                             torch.tensor([item[1] for item in x])))

Loading and preprocessing data...
Dataset shape: (25192, 42)
Class distribution:
class
normal     13449
anomaly    11743
Name: count, dtype: int64
Chronological split - Train: 15115, Test: 10077
Pretrain size: 10580, Finetune size: 4535
Creating graphs...


Created 25001/25192 graphs: 100%|██████████| 25192/25192 [00:11<00:00, 2108.50it/s]

Pretrain graphs: 10580
Finetune graphs: 4535, Labels: 4535
Test graphs: 10077, Labels: 10077


In [15]:
# Initialize the GraphEncoder and ContrastiveLoss
encoder = GraphEncoder(input_dim, hidden_dim).to(device)
contrastive_loss_fn = ContrastiveLoss().to(device)
# Initialize optimizer and learning rate scheduler for pre-training
optimizer = optim.AdamW(encoder.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs_pretrain)

# Store encoder parameter count
training_history['model_parameters']['encoder_params'] = sum(p.numel() for p in encoder.parameters())

print("\nStarting contrastive pre-training...")
# Set encoder to training mode
encoder.train()

pretrain_progress = tqdm(range(num_epochs_pretrain), desc="Pre-training", position=0, leave=True)
for epoch in pretrain_progress:
    epoch_start_time = time.time()
    total_loss = 0
    total_contrastive = 0
    total_reg = 0
    num_batches = 0
    batch_losses_epoch = []

    for batch_idx, batch in enumerate(pretrain_loader):
        batch = batch.to(device)

        # Create two augmented views of the batch
        batch_a = augmentation(batch)
        batch_b = augmentation(batch)

        batch_a = batch_a.to(device)
        batch_b = batch_b.to(device)

        # Get embeddings from the encoder for both views
        z_a = encoder(batch_a.x, batch_a.edge_index, batch_a.batch)
        z_b = encoder(batch_b.x, batch_b.edge_index, batch_b.batch)

        # Calculate contrastive loss
        loss, contrastive, reg = contrastive_loss_fn(z_a, z_b)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(encoder.parameters(), max_norm=1.0) # Gradient clipping
        optimizer.step()

        # Accumulate loss metrics
        total_loss += loss.item()
        total_contrastive += contrastive.item()
        total_reg += reg.item()
        num_batches += 1
        
        batch_losses_epoch.append(loss.item())

    # Step the learning rate scheduler
    scheduler.step()
    
    epoch_time = time.time() - epoch_start_time
    training_history['epoch_times'].append(epoch_time)

    # Calculate average losses for the epoch
    avg_loss = total_loss / num_batches
    avg_contrastive = total_contrastive / num_batches
    avg_reg = total_reg / num_batches
    
    # Store pre-training metrics in history
    training_history['pretrain_losses'].append(avg_loss)
    training_history['pretrain_contrastive_losses'].append(avg_contrastive)
    training_history['pretrain_reg_losses'].append(avg_reg)
    training_history['pretrain_learning_rates'].append(scheduler.get_last_lr()[0])
    training_history['batch_losses'].extend(batch_losses_epoch)

    # Update progress bar
    pretrain_progress.set_postfix({
        'Loss': f'{avg_loss:.4f}',
        'Contrastive': f'{avg_contrastive:.4f}',
        'Reg': f'{avg_reg:.4f}',
        'LR': f'{scheduler.get_last_lr()[0]:.6f}',
        'Time': f'{epoch_time:.2f}s'
    })


Starting contrastive pre-training...


Pre-training: 100%|██████████| 50/50 [06:00<00:00,  7.20s/it, Loss=0.2879, Contrastive=0.2783, Reg=0.9596, LR=0.000000, Time=7.09s]


In [16]:
print("\nPre-training completed. Starting fine-tuning...")

# Initialize the ClassificationHead
classification_head = ClassificationHead(hidden_dim // 2, 2).to(device)
# Store classification head parameter count
training_history['model_parameters']['classifier_params'] = sum(p.numel() for p in classification_head.parameters())
training_history['model_parameters']['total_params'] = training_history['model_parameters']['encoder_params'] + training_history['model_parameters']['classifier_params']

# Freeze all encoder parameters except for the last GIN layer
for param in encoder.parameters():
    param.requires_grad = False

for param in encoder.convs[-1].parameters():
    param.requires_grad = True
for param in encoder.batch_norms[-1].parameters():
    param.requires_grad = True

# Initialize optimizer for fine-tuning (includes classification head and last encoder layer)
finetune_optimizer = optim.AdamW(
    list(classification_head.parameters()) +
    list(encoder.convs[-1].parameters()) +
    list(encoder.batch_norms[-1].parameters()),
    lr=learning_rate * 0.1, # Use a smaller learning rate for fine-tuning
    weight_decay=1e-4
)

# Initialize learning rate scheduler for fine-tuning
finetune_scheduler = optim.lr_scheduler.StepLR(finetune_optimizer, step_size=10, gamma=0.5)
# Define cross-entropy loss for classification
criterion = nn.CrossEntropyLoss()

# Set models to training mode
encoder.train()
classification_head.train()

finetune_progress = tqdm(range(num_epochs_finetune), desc="Fine-tuning", position=0, leave=True)
best_val_auc = 0
patience = 5
patience_counter = 0

for epoch in finetune_progress:
    epoch_start_time = time.time()
    total_loss = 0
    correct = 0
    total = 0

    for batch_data, batch_labels in finetune_loader:
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)

        # Get embeddings from the encoder
        embeddings = encoder(batch_data.x, batch_data.edge_index, batch_data.batch)
        # Get classification outputs
        outputs = classification_head(embeddings)

        # Calculate classification loss
        loss = criterion(outputs, batch_labels)

        # Backpropagation and optimization
        finetune_optimizer.zero_grad()
        loss.backward()
        # Gradient clipping for fine-tuning
        torch.nn.utils.clip_grad_norm_(
            list(classification_head.parameters()) +
            list(encoder.convs[-1].parameters()) +
            list(encoder.batch_norms[-1].parameters()),
            max_norm=1.0
        )
        finetune_optimizer.step()

        # Accumulate loss and accuracy metrics
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()

    # Step the fine-tuning learning rate scheduler
    finetune_scheduler.step()
    
    epoch_time = time.time() - epoch_start_time

    # Calculate average loss and accuracy for the epoch
    avg_loss = total_loss / len(finetune_loader)
    accuracy = 100. * correct / total
    
    # Store fine-tuning metrics in history
    training_history['finetune_losses'].append(avg_loss)
    training_history['finetune_accuracies'].append(accuracy)
    training_history['finetune_learning_rates'].append(finetune_scheduler.get_last_lr()[0])

    # Evaluate on validation (finetune) set periodically
    if epoch % 5 == 0:
        val_metrics, _, _, _ = evaluate_model(encoder, classification_head, finetune_loader, device)
        training_history['validation_metrics'].append({
            'epoch': epoch,
            'metrics': val_metrics
        })

    # Update progress bar
    finetune_progress.set_postfix({
        'Loss': f'{avg_loss:.4f}',
        'Accuracy': f'{accuracy:.2f}%',
        'Time': f'{epoch_time:.2f}s'
    })


Pre-training completed. Starting fine-tuning...


Fine-tuning: 100%|██████████| 30/30 [00:52<00:00,  1.73s/it, Loss=0.0856, Accuracy=97.10%, Time=1.44s]


In [ ]:
print("\nFine-tuning completed. Evaluating on test set...")

# Set models to evaluation mode for final testing
encoder.eval()
classification_head.eval()

# Evaluate the model on the held-out test set
test_metrics, test_predictions, test_targets, test_probs = evaluate_model(encoder, classification_head, test_loader, device)

# Store test metrics in history
training_history['test_metrics'] = test_metrics
training_history['test_predictions'] = test_predictions.tolist()
training_history['test_targets'] = test_targets.tolist()
training_history['test_probabilities'] = test_probs.tolist()

print(f"\nFinal Test Results:")
print(f"AUC Score: {test_metrics['auc']:.4f}")
print(f"F1 Score: {test_metrics['f1']:.4f}")
print(f"Accuracy: {test_metrics['accuracy']:.4f}")
print(f"Precision: {test_metrics['precision']:.4f}")
print(f"Recall: {test_metrics['recall']:.4f}")

print("\nDetailed Classification Report:")
print(classification_report(test_targets, test_predictions, target_names=['Normal', 'Anomaly']))

print("\nData Leakage Verification:")
# Check for data leakage by comparing connection IDs in train and test sets
train_connection_ids = set(df.iloc[pretrain_indices + finetune_indices]['connection_id'])
test_connection_ids = set(df.iloc[test_indices]['connection_id'])
overlap = train_connection_ids.intersection(test_connection_ids)
overlap_percentage = len(overlap)/len(test_connection_ids)*100
print(f"Connection ID overlap between train and test: {len(overlap)} ({overlap_percentage:.2f}%)")

training_history['data_leakage_check'] = {
    'overlap_count': len(overlap),
    'overlap_percentage': overlap_percentage,
    'train_connection_ids': len(train_connection_ids),
    'test_connection_ids': len(test_connection_ids)
}

print(f"\nModel Summary:")
print(f"Total parameters: {training_history['model_parameters']['total_params']}")
print(f"Encoder parameters: {training_history['model_parameters']['encoder_params']}")
print(f"Classification head parameters: {training_history['model_parameters']['classifier_params']}")

# Save the complete training history to a JSON file
with open('training_history.json', 'w') as f:
    json.dump(training_history, f, indent=2)

# Prepare data for pre-training visualization and save to CSV
visualization_data = pd.DataFrame({
    'epoch': range(len(training_history['pretrain_losses'])),
    'pretrain_loss': training_history['pretrain_losses'],
    'contrastive_loss': training_history['pretrain_contrastive_losses'],
    'regularization_loss': training_history['pretrain_reg_losses'],
    'learning_rate': training_history['pretrain_learning_rates']
})

# Prepare data for fine-tuning visualization and save to CSV
finetune_data = pd.DataFrame({
    'epoch': range(len(training_history['finetune_losses'])),
    'finetune_loss': training_history['finetune_losses'],
    'finetune_accuracy': training_history['finetune_accuracies'],
    'finetune_lr': training_history['finetune_learning_rates']
})

visualization_data.to_csv('pretrain_metrics.csv', index=False)
finetune_data.to_csv('finetune_metrics.csv', index=False)

# Save test results to a CSV file
test_results_df = pd.DataFrame({
    'predictions': training_history['test_predictions'],
    'targets': training_history['test_targets'],
    'probabilities': training_history['test_probabilities']
})
test_results_df.to_csv('test_results.csv', index=False)

print("\nTraining completed successfully!")
print("Visualization data saved to:")
print("- training_history.json (complete training history)")
print("- pretrain_metrics.csv (pretraining metrics)")
print("- finetune_metrics.csv (finetuning metrics)")
print("- test_results.csv (test predictions and targets)")

print(f"\nData Collection Summary:")
print(f"- Collected {len(training_history['pretrain_losses'])} pretraining epochs")
print(f"- Collected {len(training_history['finetune_losses'])} finetuning epochs")
print(f"- Collected {len(training_history['batch_losses'])} batch losses")
print(f"- Collected {len(training_history['validation_metrics'])} validation checkpoints")
print(f"- Test set size: {len(training_history['test_predictions'])} samples")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, roc_curve, auc
import json

# Set style
plt.style.use('default')
sns.set_palette("Set2")

# Load training history
with open('training_history.json', 'r') as f:
    training_history = json.load(f)

# Create simple dashboard with key visualizations
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 1. TRAINING LOSSES
axes[0, 0].plot(training_history['pretrain_losses'], 'b-', linewidth=2, label='Pretraining')
if training_history['finetune_losses']:
    start_epoch = len(training_history['pretrain_losses'])
    finetune_epochs = range(start_epoch, start_epoch + len(training_history['finetune_losses']))
    axes[0, 0].plot(finetune_epochs, training_history['finetune_losses'], 'r-', linewidth=2, label='Finetuning')
axes[0, 0].set_title('Training Loss', fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. FINETUNING ACCURACY
axes[0, 1].plot(training_history['finetune_accuracies'], 'g-', linewidth=2, marker='o')
axes[0, 1].set_title('Finetuning Accuracy', fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy (%)')
axes[0, 1].grid(True, alpha=0.3)

# 3. TEST PERFORMANCE METRICS
test_metrics = training_history['test_metrics']
metrics_names = list(test_metrics.keys())
metrics_values = list(test_metrics.values())
bars = axes[0, 2].bar(metrics_names, metrics_values, color=['skyblue', 'lightgreen', 'orange', 'pink', 'lightcoral'])
axes[0, 2].set_title('Test Performance', fontweight='bold')
axes[0, 2].set_ylabel('Score')
axes[0, 2].set_ylim(0, 1)
# Add value labels on bars
for bar, value in zip(bars, metrics_values):
    axes[0, 2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                   f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
axes[0, 2].tick_params(axis='x', rotation=45)

# 4. CONFUSION MATRIX
cm = confusion_matrix(training_history['test_targets'], training_history['test_predictions'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', square=True, ax=axes[1, 0],
            xticklabels=['Normal', 'Anomaly'], yticklabels=['Normal', 'Anomaly'])
axes[1, 0].set_title('Confusion Matrix', fontweight='bold')
axes[1, 0].set_xlabel('Predicted')
axes[1, 0].set_ylabel('Actual')

# 5. ROC CURVE
fpr, tpr, _ = roc_curve(training_history['test_targets'], training_history['test_probabilities'])
roc_auc = auc(fpr, tpr)
axes[1, 1].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC (AUC = {roc_auc:.3f})')
axes[1, 1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', alpha=0.5)
axes[1, 1].set_xlim([0.0, 1.0])
axes[1, 1].set_ylim([0.0, 1.05])
axes[1, 1].set_xlabel('False Positive Rate')
axes[1, 1].set_ylabel('True Positive Rate')
axes[1, 1].set_title('ROC Curve', fontweight='bold')
axes[1, 1].legend(loc="lower right")
axes[1, 1].grid(True, alpha=0.3)

# 6. CLASS DISTRIBUTION
class_dist = training_history['class_distributions']['binary']
labels = list(class_dist.keys())
sizes = list(class_dist.values())
colors = ['lightblue', 'lightcoral']
axes[1, 2].pie(sizes, labels=labels, autopct='%1.1f%%', colors=colors, startangle=90)
axes[1, 2].set_title('Class Distribution', fontweight='bold')

plt.tight_layout()
plt.savefig('key_training_results.png', dpi=300, bbox_inches='tight')
plt.show()

# SIMPLE TRAINING SUMMARY
print("=" * 50)
print("TRAINING SUMMARY")
print("=" * 50)
print(f"Final Test AUC: {training_history['test_metrics']['auc']:.4f}")
print(f"Final Test Accuracy: {training_history['test_metrics']['accuracy']:.4f}")
print(f"Final Test F1 Score: {training_history['test_metrics']['f1']:.4f}")
print(f"Pretraining Loss Reduction: {training_history['pretrain_losses'][0]:.4f} → {training_history['pretrain_losses'][-1]:.4f}")
print(f"Best Finetuning Accuracy: {max(training_history['finetune_accuracies']):.2f}%")
print("=" * 50)